In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00


In [2]:
!pip install imblearn

# Data Sampling Approaches

In this notebook we experiment with vaious data sampling approaches to improve the model's performance.

Our analysis of the data showed that the dataset had imbalanced classes. The number of examples without patronising and condescending language (PCL) is much higher than the number of examples with PCL. Models trained on imbalanced datasets may learn biased prior probabilities.

In [5]:
import torch
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaModel, Trainer, TrainingArguments
from datasets import load_dataset
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

In [66]:
# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

def tokenize_sample(sample):
  sample['text'] = [str(i) for i in sample['text']]
  return tokenizer(sample['text'], truncation=True)

In [36]:
# Load model
model = RobertaModel.from_pretrained('roberta-base', num_labels=2)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
train_dataset = pd.read_csv('train_dev_data/train_set.csv')
test_dataset = pd.read_csv('train_dev_data/dev_set.csv')

## Approach 1: oversampling

Random oversampling: a random choice of minority instances are duplicated.

In [55]:
X_train = train_dataset[['text']]  # Feature columns
y_train = train_dataset['label']  # Target column

# Initialize the random oversampler
ros = RandomOverSampler(random_state=42)

# Apply oversampling
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Update the dataset with the resampled values
train_dataset = pd.DataFrame(X_resampled, columns=X_train.columns)
train_dataset['label'] = y_resampled

# Verify the oversampling result
print(train_dataset['label'].value_counts())


label
0    7581
1    7581
Name: count, dtype: int64


In [56]:
from datasets import Dataset, DatasetDict

test_dataset = test_dataset[['text', 'label']]

ds = DatasetDict()

ds['train'] = Dataset.from_pandas(train_dataset)
ds['test'] = Dataset.from_pandas(test_dataset)

In [68]:
# Tokenize dataset
tokenized_train_dataset = ds['train'].map(tokenize_sample, batched=True)
tokenized_test_dataset = ds['test'].map(tokenize_sample, batched=True)

Map:   0%|          | 0/15162 [00:00<?, ? examples/s]

Map:   0%|          | 0/2094 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_roberta')
tokenizer.save_pretrained('./fine_tuned_roberta')

In [ ]:
# Evaluate performance


## Approach 2: undersampling

Random undersampling: a random choice of majority instances are removed from the dataset.